In [ ]:
import pandas as pd

main_headers = ['sNo','sPort','rxPkts','rxBytes','rxError','txPkts','txBytes','txError','portSpeed','portStat','linkStat'] 
final_header = [] 
for i in range(1,6):
  for k in range(4):
    for j in main_headers:
      final_header.append(str(i)+str(k)+j)
final_header.extend(['d11','d12','d13','d14','d21','d22','d23','d25','d31','d32','d33','d34','d35','d41','d43','d44','d45','d52','d53','d54','d55','y'])
print(len(final_header))
print(final_header)

df = pd.read_csv('alldata.csv',names=final_header)
print(df.shape)
df.head(3)

import numpy as np

### droping unusefull columns
errors_0_cols = []
delay_0_cols = []
stats_cols = []
portSpeed_cols = []
for i in df.columns:
  if 'Error' in i:
    errors_0_cols.append(i)
  elif len(i) == 3 and 'd' in i and i[1]==i[2]:
    delay_0_cols.append(i)
  elif 'portStat' in i or 'linkStat' in i:
    stats_cols.append(i)
  elif 'portSpeed' in i:
    portSpeed_cols.append(i)
print(errors_0_cols)
print(delay_0_cols)
print(stats_cols)
print(portSpeed_cols)

sum_errors = df[errors_0_cols].values
sum_delays = df[delay_0_cols].values
print(sum_errors.shape)
print(sum_errors.sum())
print(sum_delays.sum())

delay_0_cols.extend(stats_cols)
errors_0_cols.extend(delay_0_cols)
errors_0_cols.extend(portSpeed_cols)
print(len(errors_0_cols))

df.drop(errors_0_cols,axis=1,inplace=True)
df.shape

X = df.drop('y',axis=1).values
print(X.shape)
y = df['y'].values
print(y.shape)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    stratify=y,
                                                    random_state=0)


242
['10sNo', '10sPort', '10rxPkts', '10rxBytes', '10rxError', '10txPkts', '10txBytes', '10txError', '10portSpeed', '10portStat', '10linkStat', '11sNo', '11sPort', '11rxPkts', '11rxBytes', '11rxError', '11txPkts', '11txBytes', '11txError', '11portSpeed', '11portStat', '11linkStat', '12sNo', '12sPort', '12rxPkts', '12rxBytes', '12rxError', '12txPkts', '12txBytes', '12txError', '12portSpeed', '12portStat', '12linkStat', '13sNo', '13sPort', '13rxPkts', '13rxBytes', '13rxError', '13txPkts', '13txBytes', '13txError', '13portSpeed', '13portStat', '13linkStat', '20sNo', '20sPort', '20rxPkts', '20rxBytes', '20rxError', '20txPkts', '20txBytes', '20txError', '20portSpeed', '20portStat', '20linkStat', '21sNo', '21sPort', '21rxPkts', '21rxBytes', '21rxError', '21txPkts', '21txBytes', '21txError', '21portSpeed', '21portStat', '21linkStat', '22sNo', '22sPort', '22rxPkts', '22rxBytes', '22rxError', '22txPkts', '22txBytes', '22txError', '22portSpeed', '22portStat', '22linkStat', '23sNo', '23sPort', '2

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

######################## 1 #####################################
#clf = DecisionTreeClassifier(random_state=0)
#path = clf.cost_complexity_pruning_path(X_train, y_train)
#ccp_alphas, impurities = path.ccp_alphas, path.impurities
#print(ccp_alphas, impurities)

######################### 2 ################################
#param_grid = {'ccp_alpha': np.linspace(0., 0.1, 20)}
#grid = GridSearchCV(DecisionTreeClassifier(random_state=0),param_grid=param_grid, cv=10)
#grid.fit(X_train, y_train)
#print(f"best parameters: {grid.best_params_}")

tree = DecisionTreeClassifier(ccp_alpha=0,random_state=47)
tree.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=47, splitter='best')

In [ ]:
print(f"best mean cross-validation score: {tree.score(X_train,y_train)}")
print(f"test-set score: {tree.score(X_test, y_test):.3f}")

best mean cross-validation score: 1.0
test-set score: 0.991


In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(200, 100))
tree_dot = plot_tree(tree, feature_names=df.columns,fontsize=20)
plt.show()

In [ ]:
imp = tree.feature_importances_
f = {}
for i in range(len(df.columns.drop('y'))):
  if imp[i] != 0:
    f[df.columns[i]] = imp[i]

for i in f:
  print(i,'=========>',f[i])

10txBytes =========> 0.018969185704814834
11rxBytes =========> 0.0029461513650441418
11txBytes =========> 0.09871189652613058
12rxBytes =========> 0.036948757403886666
13rxPkts =========> 0.004926619782657148
13txBytes =========> 0.00046401277795460625
21rxBytes =========> 0.004694002780939017
22rxBytes =========> 0.004491220987448833
22txBytes =========> 0.10129591511840541
23rxBytes =========> 0.05500148591904492
23txPkts =========> 0.0008791165354024748
30txBytes =========> 0.003911668984115847
31rxPkts =========> 0.10352108109177714
32rxBytes =========> 0.10683830390190853
32txPkts =========> 0.0004413165007720443
32txBytes =========> 0.013460850333657385
33rxBytes =========> 0.10803049941028081
41rxBytes =========> 0.007492133191914932
41txBytes =========> 0.004826899227194235
42txBytes =========> 0.10250685671798429
43txBytes =========> 0.016661762052945787
50txBytes =========> 0.012134179383613001
51rxPkts =========> 0.014774640933895545
51rxBytes =========> 0.017208477838546274

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

y_pred = tree.predict(X_test)
print(precision_recall_fscore_support(y_test, y_pred, average='macro'))
print(precision_recall_fscore_support(y_test, y_pred, average='micro'))
print(precision_recall_fscore_support(y_test, y_pred, average='weighted'))


'''
'micro':
Calculate metrics globally by counting the total true positives, false negatives and false positives.

'macro':
Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.

'weighted':
Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label).
This alters ‘macro’ to account for label imbalance;
it can result in an F-score that is not between precision and recall.
'''

(0.9902060707323864, 0.9903420565079902, 0.9902463650333099, None)
(0.9905992949471211, 0.9905992949471211, 0.9905992949471211, None)
(0.9907464607464607, 0.9905992949471211, 0.9906471378333199, None)


"\n'micro':\nCalculate metrics globally by counting the total true positives, false negatives and false positives.\n\n'macro':\nCalculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.\n\n'weighted':\nCalculate metrics for each label, and find their average weighted by support (the number of true instances for each label).\nThis alters ‘macro’ to account for label imbalance;\nit can result in an F-score that is not between precision and recall.\n"